## LROSE Cyclone Starter Kit Julia Notebook
This is a simple tutorial in a Jupyter notebook for users to get familiar with LROSE Cyclone

### Prerequisites

First, make sure that [Jupyter notebook](http://jupyter.readthedocs.io/en/latest/install.html) is installed.

### Workflow 

This starter kit covers a basic LROSE workflow:
1. Print data header from a NEXRAD file using RadxPrint.
2. Convert Level II data to CfRadial format using RadxConvert.
3. Display the data with CfRadial files using Hawkeye.
4. Perform coordinate transformations from the polar grid to a regular grid using Radx2Grid.

Some notes for users:
- RadxConvert also supports the conversion from Level II data to sweep format which can still be utilized by soloii users. 
- RadxConvert will name and sort the files chronologically, so we encourage users to not move the data from the original directory or rename the files.
- Users are encouraged to compile their CfRadial files as an aggregation of sweep files that make up a single volume scan.

#### The following tutorial highlights how to perform the commands, but does not cover all the options. Please see the evolving [documentation](https://nsf-lrose.github.io/docs.html) on the LROSE website for more details

Install
-------

-   Complete the free [registration](../../../software.html) and follow
    the install instructions

If you've already registered, click the link at the bottom of the registration page to take you directly to the software links.

Get some data
-------------

-   A sample NEXRAD level II file is included in the starter kit, or available
    [here](http://lrose.net/_static/cfrad.20191126_000054.663_to_20191126_000432.843_SPOL_SURprecip_SUR.nc) for download with
    this quick start notebook

If you have your own data, just replace the filename in all the
subsequent examples with your data

Inspect the data
----------------

LROSE can handle 24 radar and lidar formats (and counting). To see if
your data works with LROSE, first use RadxPrint to print the metadata.

The following instructions assume you are using the Virtual Toolbox and
the sample NEXRAD file. For native apps installed in your executable
path, just drop 'lrose -- ' or replace with the absolute path to the
location where the binaries are installed. If you are using your own
data, replace the name of the file.

Since we are using a virtual machine, we have to explicitly tell the commands where the data is located. In the notebook, we can define our path as a programming variable. This is slightly different than working in a terminal where environmental variables can be used for this purpose.


In [1]:
path = pwd();

In [4]:
run(`RadxPrint -f $path/cfrad.20191126_000054.663_to_20191126_000432.843_SPOL_SURprecip_SUR.nc`)

=============== RadxVol ===============
  version: CF-Radial-1.4
  title: 
  institution: 
  references: 
  source: 
  history: 
  comment: 
  volNum: 102
  scanName: SURprecip
  scanId(VCP): 0
--------------- RadxPlatform ---------------
  instrumentName: SPOL
  siteName: MARSHALL
  instrumentType: radar
  platformType: fixed
  primaryAxis: axis_z
  latitudeDeg: 39.9502
  longitudeDeg: -105.195
  altitudeKm: 1.748
  sensorHtAglM: 0
  frequencyHz: 2.80968e+09
  radarAntGainDbH: 44.95
  radarAntGainDbV: 45.32
  radarBeamWidthDegH: 0.92
  radarBeamWidthDegV: 0.92
  radarReceiverBandwidthMhz: -9999
--------------------------------------------
  rhiMode? N
  startTimeSecs: 2019/11/26 00:00:54
  startNanoSecs: 6.63e+08
  endTimeSecs: 2019/11/26 00:04:32
  endNanoSecs: 8.43e+08
  ray times are in increasing order
  n sweeps: 7
  n rays: 3360
  n calibs: 1
  n fields: 65
    field[0]: DBZ
    field[1]: DBZ_F
    field[2]: VEL
    field[3]: VEL_F
    field[4]: VEL_HV
    field[5]: VEL_HV_F
   

Process(`RadxPrint -f /Users/mmbell/Science/lrose/lrose-cyclone/docs/starter_kit/cfrad.20191126_000054.663_to_20191126_000432.843_SPOL_SURprecip_SUR.nc`, ProcessExited(0))

If the program returns 'File format not recognized' then it is not a
file format that LROSE currently supports. Otherwise, you should see
text output with all the metadata from your file.

If you prefer to store the printed information in a .txt file, use the command:

In [5]:
run(pipeline(`RadxPrint -f $path/cfrad.20191126_000054.663_to_20191126_000432.843_SPOL_SURprecip_SUR.nc` , stdout="$path/Level2_KAMX_20161006_1906_info.txt"))

Process(`RadxPrint -f /Users/mmbell/Science/lrose/lrose-cyclone/docs/starter_kit/cfrad.20191126_000054.663_to_20191126_000432.843_SPOL_SURprecip_SUR.nc`, ProcessExited(0))

###  Convert the NEXRAD Level II data to different format

The LROSE workflow relies on the CfRadial format for all subsequent
display and analysis. Use RadxConvert to convert your file(s) to
CfRadial. It will work on any file that is recognized by RadxPrint from
the first step through the Radx engine backend.

The default output format is CfRadial, but some other write formats are supported, like Dorade sweepfiles. If you don't choose any command line options RadxConvert will convert to CfRadial.

In [6]:
run(`RadxConvert -f $path/cfrad.20191126_000054.663_to_20191126_000432.843_SPOL_SURprecip_SUR.nc -outdir $path/output`)

Program 'RadxConvert'
Run-time 2020/01/12 15:46:47.

Copyright (c) 1992 - 2020
University Corporation for Atmospheric Research (UCAR)
National Center for Atmospheric Research (NCAR)
Boulder, Colorado, USA.

Redistribution and use in source and binary forms, with
or without modification, are permitted provided that the following
conditions are met:

1) Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

2) Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

3) Neither the name of UCAR, NCAR nor the names of its contributors, if
any, may be used to endorse or promote products derived from this
software without specific prior written permission.

4) If the software is modified to produce derivative works, such modified
software should be clearly marked, so as not to 

Process(`RadxConvert -f /Users/mmbell/Science/lrose/lrose-cyclone/docs/starter_kit/cfrad.20191126_000054.663_to_20191126_000432.843_SPOL_SURprecip_SUR.nc -outdir /Users/mmbell/Science/lrose/lrose-cyclone/docs/starter_kit/output`, ProcessExited(0))

There are many more options available for RadxConvert. To use more options you can create a parameter file that can be used for subsequent conversion steps:

In [7]:
run(pipeline(`RadxConvert -f -print_params` , stdout="$path/RadxConvert.params"))

Program 'RadxConvert'
Run-time 2020/01/12 15:47:09.

Copyright (c) 1992 - 2020
University Corporation for Atmospheric Research (UCAR)
National Center for Atmospheric Research (NCAR)
Boulder, Colorado, USA.

Redistribution and use in source and binary forms, with
or without modification, are permitted provided that the following
conditions are met:

1) Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

2) Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

3) Neither the name of UCAR, NCAR nor the names of its contributors, if
any, may be used to endorse or promote products derived from this
software without specific prior written permission.

4) If the software is modified to produce derivative works, such modified
software should be clearly marked, so as not to 

Process(`RadxConvert -f -print_params`, ProcessExited(0))

If you want to convert a file to Dorade format for use in soloii or solo3, just add the -dorade flag to the RadxConvert command or change the output format in the parameter file. We recommend migrating your workflow to CfRadial, but conversion to Dorade is supported for existing workflows.

In [11]:
run(`HawkEye -f $path/output/20191126/cfrad.20191126_000054.663_to_20191126_000432.843_SPOL_SURprecip_SUR.nc`)

Process(`HawkEye -f /Users/mmbell/Science/lrose/lrose-cyclone/docs/starter_kit/output/20191126/cfrad.20191126_000054.663_to_20191126_000432.843_SPOL_SURprecip_SUR.nc`, ProcessExited(0))

### Grid the data into Cartesian space

A common first step for working with NEXRAD data is to interpolate the
data from its native radial coordinate to a regular grid. Radx2Grid can
grid ground-based and some airborne lidar and radar data in spherical
and Cartesian space. While there are many options for Radx2Grid, a basic
gridding technique can be accomplished simply:

In [10]:
run(`Radx2Grid -f $path/output/20191126/cfrad.20191126_000054.663_to_20191126_000432.843_SPOL_SURprecip_SUR.nc -outdir $path/grid`)

Program 'Radx2Grid'
Run-time 2020/01/12 15:48:29.

Copyright (c) 1992 - 2020
University Corporation for Atmospheric Research (UCAR)
National Center for Atmospheric Research (NCAR)
Boulder, Colorado, USA.

Redistribution and use in source and binary forms, with
or without modification, are permitted provided that the following
conditions are met:

1) Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

2) Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

3) Neither the name of UCAR, NCAR nor the names of its contributors, if
any, may be used to endorse or promote products derived from this
software without specific prior written permission.

4) If the software is modified to produce derivative works, such modified
software should be clearly marked, so as not to co

Process(`Radx2Grid -f /Users/mmbell/Science/lrose/lrose-cyclone/docs/starter_kit/output/20191126/cfrad.20191126_000054.663_to_20191126_000432.843_SPOL_SURprecip_SUR.nc -outdir /Users/mmbell/Science/lrose/lrose-cyclone/docs/starter_kit/grid`, ProcessExited(0))

There are many more options available for RadxConvert. To use more options you can create a parameter file that can be used for subsequent conversion steps:

In [8]:
run(pipeline(`Radx2Grid -f -print_params` , stdout="$path/Radx2Grid.params"))

### Quick Start Complete!

If you have successfully run the previous steps then you should now have
a Cartesian gridded NetCDF file to use for further analysis called
ncf_20161006_191339.nc in the 'grid' subdirectory. From here, you can
read in the CfRadial or gridded netCDF file using Julia, Python, or
other programming language for further analysis. Continue with the
[documentation](https://nsf-lrose.github.io/docs.html) to find out more about LROSE and the many options and
parameters available in the LROSE-Cyclone software suite.
